In [24]:
#todo outliers train: Number_of_Ads,Episode_Length_minutes. test: Number_of_Ads, Episode_Length_minutes 
#todo KNN imputation if the missing rate is high.
#todo Create new features that may capture underlying patterns (e.g., interaction terms, polynomial features).
#todo create separate file for data preparation

In [25]:
##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from TargetEncoder import TargetEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [26]:
warnings.filterwarnings('ignore')

In [27]:
##
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

In [28]:
##
submission = pd.read_csv('data/sample_submission.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
print(train.shape)

(750000, 12)


In [29]:
drop_cols = ['id']
target_col = ['Listening_Time_minutes']
cat_cols = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
num_cols = [col for col in train.columns if col not in drop_cols + cat_cols + target_col]

In [30]:
train.shape

(750000, 12)

In [31]:
X = train.drop(drop_cols + target_col, axis=1)
y = train['Listening_Time_minutes']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Outliers
 

In [34]:
X_train['Episode_Length_minutes'] = np.where(X_train['Episode_Length_minutes'] > 120, 120,
                                             X_train['Episode_Length_minutes'])
X_train['Number_of_Ads'] = np.where(X_train['Number_of_Ads'] > 3, 3, X_train['Number_of_Ads'])

# Data Cleaning and Preprocessing
 

In [35]:
def target_encoder(df_train, df_test, cols, enc):
    for col in cols:
        df_train.loc[:, col] = enc.fit_transform(df_train.loc[:, col], y_train).reshape(-1)
        df_test.loc[:, col] = enc.transform(df_test.loc[:, col]).reshape(-1)
    df_train[cols] = df_train[cols].apply(pd.to_numeric, errors='coerce')
    df_test[cols] = df_test[cols].apply(pd.to_numeric, errors='coerce')

    return df_train, df_test


enc = TargetEncoder()


In [36]:
X_train, X_test = target_encoder(X_train, X_test, ['Podcast_Name', 'Episode_Title'], enc)

In [37]:
numer_type_cols = X_train.select_dtypes(include=["number"]).columns
cat_type_cols = X_train.select_dtypes(include=["object"]).columns

# Define transformers
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # Mean imputation
    ("scaler", StandardScaler())  # Standard scaling
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Combine preprocessing pipelines
preprocessor = ColumnTransformer([
    ("num", num_pipeline, numer_type_cols),
    ("cat", cat_pipeline, cat_type_cols)
])



In [38]:
# X_train = preprocessor.fit_transform(X_train)
# X_test = preprocessor.transform(X_test)

# Baseline Model:

In [39]:
# Define full pipeline with Linear Regression
baseline_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

In [40]:
# Perform Cross-Validation (5-fold)
cv_rmse = np.sqrt(-cross_val_score(baseline_pipeline, X_train, y_train,
                                   scoring="neg_mean_squared_error", cv=5))

# Train on full training set
baseline_pipeline.fit(X_train, y_train)

# Make predictions on test set
y_pred = baseline_pipeline.predict(X_test)

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print Results
print(f"Cross-Validation RMSE: {cv_rmse.mean():.4f} ± {cv_rmse.std():.4f}")
print(f"Test Set RMSE: {test_rmse:.4f}")

Cross-Validation RMSE: 13.3580 ± 0.0157
Test Set RMSE: 13.3406
